In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
import sys

from tqdm import tqdm

from resnet import ResNet18
from data_aug import collect_advs, AdvDataSet

GeForce RTX 2070 SUPER


In [2]:
if torch.cuda.is_available() == True:
    device = torch.device('cuda:0')
    print(torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
device

GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [3]:
batch_size = 128
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def load_data():    
    transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), 
                                    transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    dataset = {'train': trainset, 'test': testset}
    data_loader = {'train': train_loader, 'test': test_loader}
    return dataset, data_loader

def load_iter(data_loader, data_type):
    bar_format = '{bar:30} {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}] {desc}'
    
    if data_type == 'train':
        train_loader = data_loader['train']
        train_iter = tqdm(enumerate(train_loader), total=len(train_loader), unit_scale=batch_size, bar_format=bar_format)
        return train_iter
    elif data_type == 'test':
        test_loader = data_loader['test']
        test_iter = tqdm(enumerate(test_loader), total=len(test_loader), unit_scale=batch_size, bar_format=bar_format)
        return test_iter
    else:
        print('Data Error!!!')

In [4]:
def imshow(img):
    img[0] = img[0] * 0.2023 + 0.4914
    img[1] = img[1] * 0.1994 + 0.4822
    img[2] = img[2] * 0.2010 + 0.4465
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [5]:
def train(model):
    model.train()
    train_loss = 0
    total = 0
    correct = 0
    step = 0
    train_iter = load_iter(data_loader, 'train')
    for i, (batch, label) in train_iter:
        batch, label = batch.to(device), label.to(device)
        output = model(batch)
        
        optimizer.zero_grad()
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        train_iter.set_description(f'[{acc:.2f}% ({correct}/{total})]', True)

In [6]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc = 0.
    test_iter = load_iter(data_loader, 'test')

    for i, (batch, label) in test_iter:
        batch, label = batch.to(device), label.to(device)
        output = model(batch)
        loss = loss_function(output, label)

        test_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        test_iter.set_description(f'[{acc:.2f}%({correct}/{total})]', True)
    return acc

In [7]:
def save_model(epoch, acc, optimizer):
    global best_acc
    if acc > best_acc:
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'optimizer': optimizer.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/model_{epoch}.pth')
        best_acc = acc
        print('Saving Model...')

def load_model(name):
    state_dict = torch.load(f'./models/{name}.pth', map_location=device)
    model = ResNet18()
    model.to(device)
    model.load_state_dict(state_dict['model'])
    optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
#     optimizer.load_state_dict(state_dict['optimizer'])
    return model, optimizer

In [8]:
# Training
model = ResNet18()
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[25, 35])
best_acc = 0
dataset, data_loader = load_data()
for epoch in range(1, 51):
    print(f'Epoch {epoch}')
    train(model)
    test_acc = test(model)
    save_model(epoch, test_acc, optimizer)
    scheduler.step()

KeyboardInterrupt: 

In [9]:
# Validation
model, optimizer = load_model('baseline_3')
loss_function = nn.CrossEntropyLoss()
test(model)

NameError: name 'data_loader' is not defined

## Adversarial Training

In [10]:
model, optimizer = load_model('baseline_3')
epsilon = 0.01
dataset, data_loader = load_data()
adv_instances = collect_advs(model, data_loader, epsilon)

Files already downloaded and verified
Files already downloaded and verified


██████████████████████████████ 50048/50048 [04:06<00:00 203.28it/s] [# of Collected Adv Instances : 4524]: 


In [11]:
advset = AdvDataSet(adv_instances, need_perturb_label=True)
advset.slicing(42, 1000)
trainset = dataset['train']
advset_list = advset.concat_dataset(trainset)
len(advset)

4524

In [ ]:
# Training
model = ResNet18()
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[25, 35])
best_acc = 0
data_loader['train'] = torch.utils.data.DataLoader(advset_list[2], batch_size=batch_size, shuffle=True)
for epoch in range(1, 51):
    print(f'Epoch {epoch}')
    train(model)
    test_acc = test(model)
    save_model(epoch, test_acc, optimizer)
    scheduler.step()

In [ ]:
model_list = ['baseline_3']
model_list += [f'slice_01{i}' for i in range(4)]
for m in model_list:
    state_dict = torch.load(f'models/{m}.pth')
    print(m, state_dict['acc'])

## Hyperplane Distance

In [13]:
weight, bias = model.load_weight()
weight.shape, bias.shape

(torch.Size([10, 512]), torch.Size([10]))

In [27]:
adv_loader = torch.utils.data.DataLoader(advset, batch_size=batch_size, shuffle=True)
bar_format = '{bar:30} {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}] {desc}'
adv_iter = tqdm(enumerate(adv_loader), total=len(adv_loader), unit_scale=batch_size, bar_format=bar_format)
for i, (batch, label, perturb_label) in adv_iter:
    batch, label = batch.to(device), label.to(device)
    output = model(batch)
    temp1 = model.load_vec()[0]
    temp2 = label[0]
    temp3 = perturb_label[0]
    break

                               0/4608 [00:00<? ?it/s] 


In [30]:
def distance(a, b, c):
    a = a.cpu().detach().numpy()
    b = b.cpu().detach().numpy()
    c = c.cpu().detach().numpy()
    return np.abs(np.matmul(a, b) + c) / np.linalg.norm(a)

print(classes[temp2], classes[temp3])
distance_list = []
for i in range(10):
    distance_list.append(distance(temp1, weight[i], bias[i]))
for i, j in zip(np.argsort(distance_list), np.sort(distance_list)):
    print(classes[i], j)

deer horse
cat 0.14598477
dog 0.25544873
truck 0.5212561
frog 0.70562583
plane 0.71229625
bird 0.7585857
ship 0.80853456
car 0.81927955
deer 2.0474293
horse 2.3645995
